Preparing to submit PacBio long read data for late 2018 early 2019


In [1]:
import os
import sys
import requests
import pandas
import paramiko
import json
from IPython import display

In [2]:
from curation_common import *
from htsworkflow.submission.encoded import DCCValidator

In [3]:
PANDAS_ODF = os.path.expanduser('~/src/pandasodf')
if PANDAS_ODF not in sys.path:
    sys.path.append(PANDAS_ODF)
    from pandasodf import ODFReader

In [4]:
GCATDIR = os.path.expanduser('~/src/gcat')
if GCATDIR not in sys.path:
    sys.path.append(GCATDIR)
    import gcat

In [5]:
from htsworkflow.submission.encoded import Document
from htsworkflow.submission.aws_submission import run_aws_cp

In [6]:
# live server & control file
server = ENCODED('www.encodeproject.org')
spreadsheet_name = "ENCODE4 May2019 PacBio"

# test server & datafile
#server = ENCODED('test.encodedcc.org')
#spreadsheet_name = os.path.expanduser('~diane/woldlab/ENCODE/C1-encode3-limb-2017-testserver.ods')

server.load_netrc()
validator = DCCValidator(server)

In [7]:
award = 'UM1HG009443'

# Submit Documents

Example Document submission

This is the wrong protocol

In [8]:
smrtbell_uuid = '/documents/2297237f-dfb0-424d-a386-c350ffe8dbe3/'
smrtbell = Document(
    os.path.expanduser('~/proj/encode4-curation/ENCODE_protocol_brain_pacbio.pdf'),
    'general protocol',
    'SMRTbell cDNA library construction protocol for PacBio sequencing',
)
body = smrtbell.create_if_needed(server, smrtbell_uuid, validator)
if '@graph' in body:
    print(body['@graph'][0]['@id'])
else:
    print(body)

{'attachment': {'type': 'application/pdf', 'download': 'ENCODE_protocol_brain_illumina.pdf', 'href': '@@download/attachment/ENCODE_protocol_brain_illumina.pdf', 'md5sum': 'fa764ec1e72b089f19a3a2d35a108aa5'}, '@id': '/documents/2297237f-dfb0-424d-a386-c350ffe8dbe3/', 'status': 'in progress', 'schema_version': '8', 'actions': [{'title': 'Edit JSON', 'profile': '/profiles/Document.json', 'name': 'edit-json', 'href': '/documents/2297237f-dfb0-424d-a386-c350ffe8dbe3/#!edit-json'}, {'title': 'Edit', 'profile': '/profiles/Document.json', 'name': 'edit', 'href': '/documents/2297237f-dfb0-424d-a386-c350ffe8dbe3/#!edit'}], 'date_created': '2019-08-26T23:17:06.844139+00:00', 'award': {'title': 'TOWARD A COMPREHENSIVE FUNCTIONAL ANNOTATION OF THE HUMAN GENOME', 'viewing_group': 'ENCODE3', 'end_date': '2016-07-31', '@id': '/awards/U54HG006998/', 'status': 'disabled', 'schema_version': '7', 'start_date': '2012-09-21', 'rfa': 'ENCODE3', 'pi': '/users/a62cfec5-57a0-45ab-b943-8ca0e0057bb6/', 'url': 'ht

# Submit Annotations

In [9]:
#sheet = gcat.get_file(spreadsheet_name, fmt='pandas_excel')
#annotations = sheet.parse('Annotations', header=0)
#created = server.post_sheet('/annotations/', annotations, verbose=True, dry_run=True)
#print(len(created))

In [10]:
#if created:
#    annotations.to_excel('/tmp/annotations.xlsx', index=False)

# Register Biosamples

In [28]:
book = gcat.get_file(spreadsheet_name, fmt='pandas_excel')
biosample = book.parse('Biosamples', header=0)
created = server.post_sheet('/biosamples/', biosample, 
                            verbose=True, 
                            dry_run=True, 
                            validator=validator)
print(len(created))

0


In [27]:
if created:
    biosample.to_excel('/dev/shm/biosamples.xlsx', index=False)

# Register Libraries

In [31]:
print(spreadsheet_name)
book = gcat.get_file(spreadsheet_name, fmt='pandas_excel')
libraries = book.parse('Libraries', header=0)
created = server.post_sheet('/libraries/', 
                            libraries, 
                            verbose=True, 
                            dry_run=True, 
                            validator=validator)
print(len(created))

ENCODE4 May2019 PacBio
0


In [30]:
if created:
    libraries.to_excel('/dev/shm/libraries.xlsx', index=False)

# Register Experiments

In [36]:
print(server.server)
book = gcat.get_file(spreadsheet_name, fmt='pandas_excel')
experiments = book.parse('Experiments', header=0)
created = server.post_sheet('/experiments/', 
                            experiments, 
                            verbose=True, 
                            dry_run=True, 
                            validator=validator)
print(len(created))

www.encodeproject.org
0


In [35]:
if created:
    experiments.to_excel('/dev/shm/experiments.xlsx', index=False)

# Register Replicates

In [42]:
print(server.server)
print(spreadsheet_name)
book = gcat.get_file(spreadsheet_name, fmt='pandas_excel')
replicates = book.parse('Replicates', header=0)
created = server.post_sheet('/replicates/',
                            replicates, 
                            verbose=True, 
                            dry_run=True, 
                            validator=validator)
print(len(created))

www.encodeproject.org
ENCODE4 May2019 PacBio
0


In [37]:
if created:
    replicates.to_excel('/dev/shm/replicates.xlsx', index=False)

# What are the biosamples?

Are the libraries linked reasonably?

In [ ]:
pb82 = server.get_json('ali-mortazavi:PB82')
print(pb82['@id'], pb82['aliases'])
print(pb82['biosample']['@id'], pb82['biosample']['summary'])
pb82_rep = server.get_json(pb82['replicates'][0])
print(pb82_rep['biological_replicate_number'], pb82_rep['technical_replicate_number'])

In [ ]:
pb83 = server.get_json('ali-mortazavi:PB83')
print(pb83['@id'], pb83['aliases'])
print(pb83['biosample']['@id'], pb83['biosample']['summary'])
pb83_rep = server.get_json(pb83['replicates'][0])
print(pb83_rep['biological_replicate_number'], pb83_rep['technical_replicate_number'])

In [ ]:
pb84 = server.get_json('ali-mortazavi:PB84')
print(pb84['@id'], pb84['aliases'])
print(pb84['biosample']['@id'], pb84['biosample']['summary'])
pb84_rep = server.get_json(pb84['replicates'][0])
print(pb84_rep['biological_replicate_number'], pb84_rep['technical_replicate_number'])

In [ ]:
pb85 = server.get_json('ali-mortazavi:PB85')
print(pb85['@id'], pb85['aliases'])
print(pb85['biosample']['@id'], pb85['biosample']['summary'])
pb85_rep = server.get_json(pb85['replicates'][0])
print(pb85_rep['biological_replicate_number'], pb85_rep['technical_replicate_number'])